In [ ]:
from utils import ltf_to_sent
import tqdm
import csv
import os
import spacy
import tqdm
nlp = spacy.load("en_core_web_trf")

#### Run training

cd spacy_train

python -m spacy train ./config.cfg --output ./output_trained_pipeline --verbose --gpu-id 0

# Main file

Spacy 

* NER 101: https://spacy.io/usage/linguistic-features#named-entities
* Training: https://spacy.io/usage/training
* Spacy-101: https://spacy.io/usage/spacy-101
* NER architectures: https://spacy.io/api/architectures#parser

In [ ]:
from utils import ltf_to_sent
import tqdm
import csv
import os
import spacy
import tqdm
nlp = spacy.load("en_core_web_trf")

def get_raw_text_from_file(filepath):
  # /home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/training/data/ltf/20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2
    basepath = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/training/data/ltf/'
    filepath = os.path.join(basepath, filepath)
    filepath += ".ltf.xml"
    text_file = open(filepath, "r", encoding="utf-8")
    ltf_str = text_file.read()
    text = ltf_to_sent.ltf_to_plaintext(ltf_str)
    return text
      
def get_gold_labels():
  # gold_labels_path = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/training/annotations/edl.tab'
  # gold_labels_path = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/kas_source/utils/edl.tab'
  gold_labels_path = './edl.tab'
  lines = []
  with open(gold_labels_path, encoding="utf-8") as tsv:
    for line in csv.reader(tsv, dialect="excel-tab"): # dialect="excel-tab" or delimiter="\t" 
      lines.append(line)
  print("example lline:", lines[0])
  return lines


def main():
  labels = get_gold_labels()
  
  # init variables
  first_doc_name = labels[0][3].split(':')[0]
  last_doc_filename = first_doc_name
  count = 0 
  num_span_is_none = 0
  doc = nlp(get_raw_text_from_file(first_doc_name))
  doc.ents = []
  db = spacy.tokens.DocBin()

  
  # iterate thru all labels, saving docs as we go
  for _, num, span_text, doc_name, ent_gold, annot_list, NAM_type, const in tqdm.tqdm(labels):
    curr_doc_filename = doc_name.split(':')[0]
    if curr_doc_filename != last_doc_filename:
      # save the last spacy.doc, and load in the next spacy.doc. 
      last_doc_filename = curr_doc_filename
      db.add(doc)
      doc = nlp(get_raw_text_from_file(curr_doc_filename))
      doc.ents = []
      print("starting new doc")
      print("Doc: ", doc)
      return
    
    # iterate thru the labels and set the spans.
    start_char, end_char = doc_name.split(':')[1].split('-')[0], doc_name.split(':')[1].split('-')[1]

    print("text:", span_text)
    print("ent_gold:", ent_gold)
    print("annot_list:", annot_list)
    print(doc_name)
    print(curr_doc_filename)
    print(start_char, end_char)

    print("Doc ents:", doc.ents)
    
    span = doc.char_span(int(start_char), int(end_char)+1, label=annot_list, alignment_mode="contract")
    print("span:",span)
    print("span type:", type(span))
    if span is None:
      num_span_is_none += 1
      continue
    if type(doc.ents) is not "NoneType":
      doc.ents = list(doc.ents) + [span]
    else:
      doc.ents = [span]
    
    if count > 3:
      break
    count += 1
  # end loop 
  db.to_disk("./train_concat_annot_list.spacy")
  print("⚠️ num_span_is_none:",num_span_is_none)

main()



In [ ]:


""" Construct TEST DATA .spacy """


from utils import ltf_to_sent
import tqdm
import csv
import os
import spacy
import tqdm
nlp = spacy.load("en_core_web_trf")

def print_test_file():      
  db = spacy.tokens.DocBin()
  basepath = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/training/data/ltf/'
  for file in tqdm.tqdm(os.listdir(basepath)):
    if file.endswith(".ltf.xml"):
      filepath = os.path.join(basepath, file)
      # print(filepath)
      text_file = open(filepath,"r",encoding="utf-8")
      ltf_str = text_file.read()
      raw_text = ltf_to_sent.ltf_to_plaintext(ltf_str)
      print(raw_text)
    break


def make_test_docs():      
  db = spacy.tokens.DocBin()
  basepath = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/test/data/ltf/'
  for file in tqdm.tqdm(os.listdir(basepath)):
    if file.endswith(".ltf.xml"):
      filepath = os.path.join(basepath, file)
      # print(filepath)
      text_file = open(filepath,"r",encoding="utf-8")
      ltf_str = text_file.read()
      raw_text = ltf_to_sent.ltf_to_plaintext(ltf_str)
      # print(raw_text)
      doc = nlp(raw_text)
      doc.ents = []
      db.add(doc)

  db.to_disk("./test_docs.spacy")
  return db

print_test_file()
return
make_test_docs()


In [9]:
test_text = """Ebola survivor Kent Brantly donates blood to help treat NBC cameraman Ashoka Mukpo Wednesday, October 8, 2014 10:13:08 GMT-05:00 The Nebraska hospital treating Ebola patient Ashoka Mukpo said that he is scheduled on Wednesday to receive the same convalescent serum from the same donor – Ebola survivor Kent Brantly – as American doctor Rick Sacra received several weeks ago. Mukpo, a freelance cameraman who was diagnosed with Ebola while working for the network in Liberia, will receive some of Brantly's blood as part of his treatment against the virus at the Nebraska Medical Center. In September, Brantly, an American doctor, donated some of his plasma to Sacra, who was also treated at the Nebraska Medical Center. In what the hospital called “an amazing stroke of luck,” Brantly was traveling through the Midwest on Tuesday on his way to Texas. He is scheduled to speak later this week at his alma mater, Abilene Christian University, according to Samaritan’s Purse, the international relief organization for which Brantly was working when he contracted Ebola at a missionary hospital in Liberia. Brantly later was flown to Atlanta for treatment and released in August. Brantly was driving through Kansas City on Tuesday when he received a call from Angela Hewett, associate medical director of the biocontainment unit at the Nebraska Medical Center, where Sacra was treated and where Mukpo currently is being treated. Remarkably, Brantly’s blood type matched Mukpo’s blood type, just as it had with Sacra. Brantly stopped at a facility in Kansas City to give his blood, which was then flown to Omaha. “It’s not a likely scenario that he would again have the same blood type,” Hewett said in a statement released by the hospital. “We are incredibly grateful that that Dr. Brantly would take the time to do this, not once, but twice.” In a statement to NBC, Mukpo's father, Mitchell Levy, called Brantly's donation an act of "kindness and generosity" that "makes me believe in the goodness of humanity." The theory behind the transfusion treatment is this: The blood of an Ebola survivor should carry antibodies of the virus. By giving a current Ebola patient an injection of a compatible amount of blood plasma from someone who recovered from the disease, those antibodies could help the patient fight the virus. In September, a panel of World Health Organization experts said such transfusions were a promising direction for experimental treatments against Ebola, and one that officials working in West Africa should pursue. Like many Westerners who have contracted the illness, Brantly, Sacra and Mukpo all received experimental treatments -- often more than one -- against Ebola. Brantly, the first Westerner repatriated to the United States for treatment during the deadliest Ebola outbreak in history, received the blood of a young Ebola survivor he'd treated in Liberia, along with the experimental drug zMapp. Sacra was treated with Brantly's blood and the drug TKM-Ebola. Mukpo is receiving an experimental drug called brincidofovir. It is the same drug given in Dallas to Thomas Eric Duncan, the Liberian man who was diagnosed with Ebola after traveling to the United States. Duncan died on Wednesday. A spokesman for Samaritan’s Purse on Wednesday said Brantly had been contacted by the Dallas hospital treating Duncan to see if he would also be willing to donate blood to him, should their blood types match. Brantly said he definitely would be willing to do so but never heard back after that. “The assumption is they weren’t a match,” said Samaritan’s Purse spokesman Jeremy Blume. “He was never contacted again.” More than 3,800 people have died from the Ebola epidemic in West Africa, according to the WHO, and there have been more than twice as many total cases reported in the region. [this post has been updated] RELATED: Spanish nurse with Ebola may have touched her face with a glove Spanish case highlights the risks and benefits of repatriating Ebola patients
"""
train_text = """In India, Syrian regime courts BRIC countries Friday, March 8, 2013 14:05:43 GMT-05:00 The bloc of the five emerging economies called BRICS may have lost some of its sheen recently, but it is being wooed as an important lobbying group by the besieged Syrian government. During her three-day visit to India, senior Syrian minister Bouthaina Shaaban asked New Delhi to take the lead in drafting a strong statement in support of Syria, when the five nations -- comprised of Brazil, Russia, India, China and South Africa -- meet at a conference later this month. Shaaban said she had handed Indian officials a letter from the Syrian President Bashar al-Assad addressed to Indian Prime Minister Manmohan Singh. The letter, she said, asked India to call for a peaceful resolution that did not impinge on Syria’s sovereignty. “We want India, Brazil, Russia, China and South Africa to make a very strong decision to support a political solution in Syria, to support the right of the Syrian people to decide a future for themselves,” Shaaban, the political and media advisor to Assad, told reporters in New Delhi on Friday. Shaabansaid that the armed anti-government rebels were religious fundamentalists, and criticized Western nations for supporting the fighters. “Thank God, there are countries like the BRICS… to introduce reason into what is happening in the international community,” Shabaan said. “Otherwise we would have faced what Libya is facing today. BRICS have been taking a very honest and very balanced stance.” In the past two years, Syria has been rocked by a bloody revolt that has killed tens of thousands of people. China and Russia have opposed moves by the the United States and other western nations to impose sanctions to pressure the Syrian government. Last year, the BRICS nations called for an end to the rhetoric of military action against Syria. India has until now walked the tightrope between the United States and Syria. It voted in favor of sanctions, but later abstained from another vote in the United Nations General Assembly, saying it opposed acts that aimed at change of regime in Syria. On Wednesday, India’s foreign office expressed its “deep concern on the security situation in Syria” and said that the Geneva Communique, which had called for respecting Syria’s sovereignty, must form the basis for a solution. On Friday, Shaaban also urged reporters to use the term “international community” with caution. “The BRICS is also a big part of the international community, so please stop using the term when you are referring to Western forces,” Shaaban said. Then she added, “It is very difficult to counter Western narrative.”
"""

In [10]:
# INFERENCE

custom_model = spacy.load("./spacy_tain_concat_annot_list/output_trained_pipeline/model-best")
# test_doc = custom_model("The U.S. Farmers are on strike again.")
# test_doc = custom_model(test_text)


In [12]:
test_doc = custom_model(test_text)
for ent in test_doc.ents:
    print(ent, ent.label_)

Ebola IllHealth;IllHealth.Disease;IllHealth.Disease.CommunicableDisease
survivor PER;PER.SituationalRole.Patient
Kent Brantly PER;PER.Executive;PER.Executive.AcademicAdministrator
NBC Document;Document.Report
cameraman PER;PER.Professional;PER.Professional.Journalist
Ashoka Mukpo PER
Nebraska GPE;GPE.City
hospital FAC;FAC.Building;FAC.Building.Hospital
Ebola IllHealth;IllHealth.Disease;IllHealth.Disease.CommunicableDisease
patient PER;PER.SituationalRole.Patient
Ashoka Mukpo PER
he PER;PER.SituationalRole.Patient
donor PER;PER.SituationalRole.Patient
– Ebola ORG;ORG.Government;ORG.Government.LegislativeCommittee
survivor PER;PER.SituationalRole.Patient
Kent Brantly – as American doctor Rick Sacra received PER;PER.Executive;PER.Executive.AcademicAdministrator
Mukpo PER;PER.SituationalRole.Patient
cameraman PER;PER.Professional;PER.Professional.Journalist
who PER;PER.SituationalRole.Patient
Ebola IllHealth;IllHealth.Disease;IllHealth.Disease.CommunicableDisease
network ORG
Liberia GPE;GP

In [7]:
""" Print a doc's ents. """
path = "train_concat_annot_list.spacy"

db = spacy.tokens.DocBin()
docs = db.from_disk(path)

# for doc in docs.get_docs(custom_model.vocab):
for doc in docs.get_docs(nlp.vocab):
    print(doc.ents)
    # doc = custom_model(doc)
    # print(do)
    # doc = nlp(doc)
    for ent in doc.ents:
        print(ent, '--', ent.label_)
    break


(Farmers, U.S., farmers, They, Bloomberg, survey, farmers, bankers, analysts, New Jersey, growers, fields, land, Midwest, states, farmland, Agriculture Department, China, Canada, U.N., data, Todd Wachtel, who, Illinois, his, cornfields, Farmers, they, United States, world, exporter, Brazil, Argentina, Agriculture Department, its, U.S., Farmers, ethanol, acres, fields, acres, fifth, acres, survey, U.S., farmers, Goldman Sachs, analysts, commodity broker, Allendale, Paul Meyers, Foresight Commodities Services, We, Standard & Poor, Agriculture Department, its, agency, Farmers, Midwest, region, areas, holder, Alan Tiemann, who, his, farm, Seward, you, your, Chicago, data)
Farmers -- PER;PER.Farmer
U.S. -- GPE;GPE.Country
farmers -- PER;PER.Farmer
They -- PER;PER.Farmer
Bloomberg -- ORG;ORG.CommercialOrganization
survey -- Document;Document.Report
farmers -- PER;PER.Farmer
bankers -- PER;PER.Professional
analysts -- PER;PER.Professional
New Jersey -- GPE;GPE.ProvinceState
growers -- PER;PER

In [ ]:
""" Run thru custom model. """



path = "./test_docs.spacy"
path = "spacy_train/train_data_with_ents.spacy"

db = spacy.tokens.DocBin()
docs = db.from_disk(path)

# for doc in docs.get_docs(custom_model.vocab):
for doc in docs.get_docs(nlp.vocab):
    print(doc.ents)
    # doc = custom_model(doc)
    # print(do)
    # doc = nlp(doc)
    for ent in doc.ents:
        print(ent, '--', ent.label_)
    break


In [ ]:
""" DEPRICATED::  THIS IS THE BEST WAY TO READ THE LTF FILES. """

# list all files in the directory
basepath = '/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/data/training/data/ltf/'
import os
for file in os.listdir("/home/rohit/Downloads/ltf_files/"):
  if file.endswith(".ltf.xml"):
    filepath = os.path.join(basepath, file)
    text_file = open(filepath,"r")
    ltf_str = text_file.read()
    
    ### Read in plain text
    text = ltf_to_sent.ltf_to_plaintext(ltf_str)
    start_token = 1009
    end_token = 1013
    print(text[start_token : end_token+1]) # NOTE the +1

In [ ]:
"""
# Read in tokens (not necessary)
token_list = ltf_to_sent.load_ltf(ltf_str)
"""

In [ ]:
""" Example code """

db = DocBin() # create a DocBin object

for text, annot in tqdm.tqdm(train): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)


In [ ]:
db.to_disk("./train.spacy") # save the docbin object

### STRETCH GOAL NOT REQUIRED: Parse mention.tab

We DON'T do localization. So just iterate thru the test set and classify each token in the `mention.tab` file into one of the `ontology_types.txt` classes. 

```
mention_path = "/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/kas_source/mentions.tab"

ontology_types_path = "/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/kas_source/utils/rufes/tools/rufes-evaluation/example/data/ontology_types.txt"
```

In [ ]:
import csv
# mention type: “NAM” (for name mentions), “NOM” (for nominal mentions), or “PRO” (for pronominal mentions)
# We don'
mention_path = "/home/kastan/githubs/UIUC-NLP-Course-Projects/HW3_NER/kas_source/mentions.tab"
lines = []
with open(mention_path) as tsv:
  for line in csv.reader(tsv, dialect="excel-tab"): #You can also use delimiter="\t" rather than giving a dialect.
    lines.append(line)
print("example lline:", lines[0])

# return
# third field
# Field 7: mention type: “NAM” (for name mentions), “NOM” (for nominal mentions), or “PRO” (for pronominal mentions)
# NIST	annotation-0	Farmers	20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:0-6	U.S. farmers	PER;PER.Farmer	NOM	1.0
count = 0 
for _, num, text, doc_name, ent_gold, annot_list, NAM_type, const in tqdm.tqdm(lines):

# Parse edl.tab (gold training labels)

In [ ]:

# Field 7: mention type: “NAM” (for name mentions), “NOM” (for nominal mentions), or “PRO” (for pronominal mentions)
# NIST	annotation-0	Farmers	20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6d6c2:0-6	U.S. farmers	PER;PER.Farmer	NOM	1.0
  ''' add ents to doc '''
  # doc = nlp.make_doc(text)
  # ents = []
  # for start, end, label in annot["entities"]:
  #   span = doc.char_span(start, end, label=label, alignment_mode="contract")
  #   if span is None:
  #     print("Skipping entity")
  #   else:
  #     ents.append(span)
  # doc.ents = ents
  # db.add(doc)
  